
# Single-Precision A·X Plus Y (SAXPY)
Part of Basic Linear Algebtra Subroutines (BLAS) Library

$$
z = αx + y
$$
$x,y,z$: vector

$α$: scalar

## 10.1 An explanation is given of what is the difference between cupy and numpy. How does cupy handle data transfers between cpu and gpu?

CuPy is an open-source array library for GPU-accelerated computing with Python. CuPy utilizes CUDA Toolkit libraries including cuBLAS, cuRAND, cuSOLVER, cuSPARSE, cuFFT, cuDNN and NCCL to make full use of the GPU architecture. 

CuPy is a GPU array backend that implements a subset of NumPy interface. NumPy runs on CPU

- data transfers between cpu and gpu:

Moving a device array to the host can be done by `cupy.asnumpy()` as follows:

    x_gpu = cp.array([1, 2, 3])  # create an array in the current device
    x_cpu = cp.asnumpy(x_gpu)  # move the array to the host.

We can also use `cupy.ndarray.get()`:

    x_cpu = x_gpu.get()

## 10.2 Function saxpy that runs on GPU using cupy is provided

In [ ]:
!nvcc --version

In [ ]:
pip install cupy

In [ ]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
import timeit

In [ ]:
ALPHA = 10
SIZE = 1e8

In [ ]:
def saxpy(a, x, y):
    return (a * x) + y

### Numpy

In [ ]:
x = np.random.rand(int(SIZE)).astype(np.float32)
y = np.random.rand(int(SIZE)).astype(np.float32)

In [ ]:
time_np = %timeit -o -r 3 -n 100 saxpy(ALPHA, x, y)

### Cupy

In [ ]:
saxpy_kernel = cp.ElementwiseKernel(
   'float32 x, float32 y, float32 a',
   'float32 z',
   'z = a * x + y',
   'saxpy_kernel')

In [ ]:
x_cp = cp.array(x)
y_cp = cp.array(y)
time_cp = %timeit -o -r 3 -n 100 saxpy(ALPHA, x_cp, y_cp)

In [ ]:
time_cp_kernel = %timeit -o -r 3 -n 100 saxpy_kernel(ALPHA, x_cp, y_cp)

## 10.3 Graph (OX - size of arrays, OY - computation time) is given. Plot numpy and cupy implementations

In [ ]:
from tqdm import tqdm
t_np_arr = np.zeros(9)
t_cp_arr = np.zeros(9)
t_cp_kernel_arr = np.zeros(9)
for i in tqdm(range(9)):
    size = 10**i
    x = np.random.rand(size).astype(cp.float32)
    y = np.random.rand(size).astype(cp.float32)
    t_np = %timeit -o -q -r 3 -n 100 saxpy(ALPHA, x, y)
    t_np_arr[i] = t_np.best
    
    x = cp.array(x)
    y = cp.array(y)
    t_cp = %timeit -o -q -r 3 -n 100 saxpy(ALPHA, x, y)
    t_cp_arr[i] = t_cp.best
    
    t_cp_k = %timeit -o -q -r 3 -n 100 saxpy_kernel(ALPHA, x, y)
    t_cp_kernel_arr[i] = t_cp_k.best

In [ ]:
time_arr = [10**i for i in range(9)]
plt.figure(figsize=(14, 8))
plt.plot(time_arr, t_np_arr, label='Numpy')
plt.plot(time_arr, t_cp_arr, label='Cupy')
plt.plot(time_arr, t_cp_kernel_arr, label='Cupy Kernel')
plt.yscale('log')
plt.xscale('log')
plt.legend(fontsize=20)
plt.xlabel('Array size',fontsize=20)
plt.ylabel('Computation time',fontsize=20)
plt.xticks(time_arr,fontsize=20)
plt.yticks(fontsize=20)
plt.show()